# FastOptSolver - Optimization Algorithm Comparison

## Objective
Compare the performance of ISTA and FISTA algorithms on the housing dataset.

## ISTA/FISTA Theory

ISTA (Iterative Shrinkage-Thresholding Algorithm) is a first-order optimization method for solving problems with non-smooth regularizers. It has a convergence rate of O(1/k).

FISTA (Fast Iterative Shrinkage-Thresholding Algorithm) is an accelerated version of ISTA with a convergence rate of O(1/k²). It uses a momentum term to accelerate convergence while maintaining the simplicity of ISTA.

In [ ]:
from experiments.housing_benchmark import run_ista_vs_fista_on_housing
run_ista_vs_fista_on_housing(n_iter=100)

## Conclusion and Next Steps

From the results, we can observe the convergence behavior of ISTA vs FISTA on the housing dataset.

Next steps could include:
- Trying different regularization parameters
- Comparing with other optimization methods (e.g., ADMM)
- Testing on different datasets